<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-packages-and-data" data-toc-modified-id="Load-packages-and-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load packages and data</a></span></li><li><span><a href="#Predictive-models" data-toc-modified-id="Predictive-models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Predictive models</a></span><ul class="toc-item"><li><span><a href="#SVM-with-RBF-Kernel" data-toc-modified-id="SVM-with-RBF-Kernel-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>SVM with RBF-Kernel</a></span><ul class="toc-item"><li><span><a href="#With-smaller-subset-of-data" data-toc-modified-id="With-smaller-subset-of-data-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>With smaller subset of data</a></span><ul class="toc-item"><li><span><a href="#Hyperopt" data-toc-modified-id="Hyperopt-2.1.1.1"><span class="toc-item-num">2.1.1.1&nbsp;&nbsp;</span>Hyperopt</a></span></li><li><span><a href="#With-more-data" data-toc-modified-id="With-more-data-2.1.1.2"><span class="toc-item-num">2.1.1.2&nbsp;&nbsp;</span>With more data</a></span></li></ul></li></ul></li></ul></li></ul></div>

## Load packages and data

In [1]:
import pdb 
import glob
import copy
import math
import pickle

import numpy as np
import pandas as pd
import scipy as sp

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
 
import missingno  # for visualizing missing data

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, \
    GridSearchCV, ShuffleSplit

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, \
    LogisticRegressionCV, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, classification_report, \
    precision_recall_curve, average_precision_score, f1_score, \
    roc_curve, auc, roc_auc_score, make_scorer,\
    accuracy_score, balanced_accuracy_score

from sklearn.externals import joblib
from sklearn.utils import resample
from sklearn.utils.fixes import signature


# Set up pandas table display
pd.set_option('display.width', 120)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

# Set plotting options
sns.set() # Use seaborn defaults for plotting
%matplotlib inline 


# Adjust number of CPU cores to use
# n_jobs=2

In [2]:
# Load preprocessed training and test set, incl. feature names 

# X_train = joblib.load('data_processed/X_train.joblib')
# X_test = joblib.load('data_processed/X_test.joblib')
# y_train = joblib.load('data_processed/y_train.joblib')
# y_test = joblib.load('data_processed/y_test.joblib')
X_train_small = joblib.load('data_processed/X_train_small.joblib')
X_test_small = joblib.load('data_processed/X_test_small.joblib')
y_train_small = joblib.load('data_processed/y_train_small.joblib')
y_test_small = joblib.load('data_processed/y_test_small.joblib')
feature_names = joblib.load('data_processed/feature_names')
feature_names_small = joblib.load('data_processed/feature_names_small')

## Predictive models

### SVM with RBF-Kernel

#### With smaller subset of data

In [ ]:
svm_rbf_1 = SVC(kernel='rbf', probability=False, 
              cache_size=5000, class_weight='balanced')
# Parameters to search over
param_grid = {'C': np.logspace(-3, 3, 8),
              'gamma': np.logspace(-5, 1, 8)}
# Define indices for validation split (instead of proper cross-validation)
split_indices = ShuffleSplit(n_splits=1, test_size=.2, random_state=1)

svm_rbf_gs_1 = GridSearchCV(svm_rbf_1, param_grid=param_grid,
                          return_train_score=True,
                          scoring='average_precision', cv=split_indices)
svm_rbf_gs_1.fit(X_train_small, y_train_small)

# Save model
joblib.dump(svm_rbf_gs_1, 'saved_models/svm_rbf_gs_1.joblib')

In [ ]:
# Load model
svm_rbf_gs_1 = joblib.load('saved_models/svm_rbf_gs_1.joblib')

In [ ]:
# Prediction of class
y_pred_svm_rbf_1 = svm_rbf_gs_1.predict(X_test_small)

# Distance from separating hyperplane
y_pred_distance_svm_rbf_1 = svm_rbf_gs_1.decision_function(X_test_small)

# Save results
average_precision_1['SVM (RBF Kernel)'] = \
    average_precision_score(y_test_small, y_pred_distance_svm_rbf_1)

classification_reports_1['SVM (RBF Kernel)'] = \
    classification_report(y_test_small, y_pred_svm_rbf_1)

In [ ]:
gs_best_result(svm_rbf_gs_1)

In [ ]:
gs_heatmap(svm_rbf_gs_1) #, x_digits=1, x_scientific_notation=False)

In [ ]:
pd.DataFrame(svm_rbf_gs_1.cv_results_)

##### Hyperopt

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Define hyperparameter search space
space = {
    'C': hp.uniform('C', 1E-10, 1E10),
    'gamma': hp.uniform('gamma', 1E-5, 1E8) 
}

# Function to minimize
N_FOLDS=3
N_JOBS=-2
def objective(params, n_folds=N_FOLDS, n_jobs=N_JOBS):
    """Objective function to minimize"""
    
    # Instantiate estimator
    clf = SVC(kernel='rbf', cache_size=6000, class_weight='balanced',
              random_state=1, **params)
    
    # Cross-validation object
    cv = StratifiedKFold(n_folds, random_state=1)
    
    # Compute average precision through CV
    score = cross_val_score(clf, X_train_small, y_train_small, cv=cv,
                            scoring='average_precision', n_jobs=n_jobs)
    
    # Return results
    return {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}

trials = Trials()

# Minimize objective
MAX_EVALS=4 
best = fmin(objective, space, algo=tpe.suggest,
            max_evals=MAX_EVALS, trials=trials)

print ('Best parameters: {}'.format(best))

In [ ]:
# Get the values of the optimal parameters
best_params = space_eval(space, best)

# Fit the model with the optimal hyperparamters
clf.set_params(**best_params)
clf.fit(X_train_small, y_train_small);

# Score with the test data
y_score = pipe.predict_proba(X_test_small)
ap_score = average_precision_scorerecision_score(y_test_small, y_score[:,1])

In [ ]:
svm_rbf_1score(X_test_small, y_test_small)

In [ ]:
print(svm_rbf_1best_model())

In [ ]:
svm_rbf_1(X_test_small)

In [ ]:
# params = {'objective':'binary:logistic', 'eval_metric':'map',
#          'scale_pos_weight':5,  # Balance class weight
#          'seed':0, 'learning_rate':0.1, 'n_estimators':1000,
#          'max_depth':5, 'min_child_weight':1, 'gamma':0,
#          'subsample':0.8, 'colsample_bytree':0.8,
#          'nthread':4}
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'binary:logistic',
    'eval_metric':'map'
}

xgb_cv = xgb.train(params=params, dtrain=data_xgb_train, 
                num_boost_round=1000,
                evals=[(data_xgb_test, "test")],
                early_stopping_rounds=50) 
#                 stratified=True, num_boost_round=1000, 
#                 early_stopping_rounds=50, verbose_eval=False,
#                 as_pandas=True, seed=0)

In [ ]:
# Parameters to search over
param_grid = {'max_depth'= [3, 5, 7, 10],  # Control complexity
              'min_child_weight'= , # The higher, the more regularization
              'gamma'= 0, # Higher value leads to fewer splits for a given node (i.e. more regularization) if
              'subsample'= [0.5, 0.75, 1],  # Fraction of observations per tree 
              'colsample_bytree': [0.5, 0.75, 1]} # Fraction of features per tree
# Grid search
xgb_gs_1 = GridSearchCV(xgb_1, param_grid=param_grid, 
                        num_boo
                        scoring='average_precision',
                        return_train_score=True,
                        n_jobs=n_jobs, cv=3, verbose=5)
xgb_gs_1.fit(X_train_small, y_train_small) 

# Save model
joblib.dump(xgb_gs_1, 'saved_models/xgb_gs_1.joblib')

# Predictions
y_proba_xgb = xgb_.predict_proba(X_test_small)[:, 1]
average_precision_score(y_test_small, y_proba_xgb)

In [ ]:
xgb_1 = XGBClassifier(objective='binary:logistic', learning_rate=0.1,
               n_estimators=1000,
               max_depth=5,min_child_weight=1, gamma=0,
               subsample=0.8, colsample_bytree=0.8,
               seed=0, nthread=n_jobs)
xgb_1.fit(X_train_small, y_train_small,
          early_stopping_rounds=50,
          eval_set=[(X_test_small, y_test_small)],
          eval_metric='map'
         )

In [ ]:
# Save model
# joblib.dump(xgb_gs_1, 'saved_models/xgb_gs_1.joblib')

# Predictions
y_proba_xgb = xgb_1.predict_proba(X_test_small)[:, 1]
average_precision_score(y_test_small, y_proba_xgb)

In [ ]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)

In [ ]:
params = {'objective':'binary:logistic', 'eval_metric':'map',
         'scale_pos_weight':5,  # Balance class weight
         'seed':0}
xgb_cv = xgb.cv(dtrain=data_xgb_train, params=params, nfold=3,
                num_boost_round=50, as_pandas=True)

In [ ]:
y_xgb_cv = xgb_cv.predict(X_test_small)
average_precision_score(y_test_small, y_xgb_cv)

In [ ]:
import hpsklearn
dir(hpsklearn)

In [ ]:
svm_rbf_1 = SVC(kernel='rbf', probability=False, 
              cache_size=5000, class_weight='balanced')
# Parameters to search over
param_grid = {'C': np.logspace(-3, 3, 8),
              'gamma': np.logspace(-5, 1, 8)}
# Define indices for validation split (instead of proper cross-validation)
split_indices = ShuffleSplit(n_splits=1, test_size=.2, random_state=1)

svm_rbf_gs_1 = GridSearchCV(svm_rbf_1, param_grid=param_grid,
                          return_train_score=True,
                          scoring='average_precision', cv=split_indices)
svm_rbf_gs_1.fit(X_train_small, y_train_small)

# Save model
joblib.dump(svm_rbf_gs_1, 'saved_models/svm_rbf_gs_1.joblib')

In [ ]:
# Load model
svm_rbf_gs_1 = joblib.load('saved_models/svm_rbf_gs_1.joblib')

In [ ]:
# Prediction of class
y_pred_svm_rbf_1 = svm_rbf_gs_1.predict(X_test_small)

# Distance from separating hyperplane
y_pred_distance_svm_rbf_1 = svm_rbf_gs_1.decision_function(X_test_small)

# Save results
average_precision_1['SVM (RBF Kernel)'] = \
    average_precision_score(y_test_small, y_pred_distance_svm_rbf_1)

classification_reports_1['SVM (RBF Kernel)'] = \
    classification_report(y_test_small, y_pred_svm_rbf_1)

In [ ]:
gs_best_result(svm_rbf_gs_1)

In [ ]:
gs_heatmap(svm_rbf_gs_1) #, x_digits=1, x_scientific_notation=False)

In [ ]:
pd.DataFrame(svm_rbf_gs_1.cv_results_)

##### With more data